In [1]:
from transformers import BertTokenizer

# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def preprocess_email(email):
    # Tokenize the input email text
    encoding = tokenizer(email, truncation=True, padding=True, max_length=512, return_tensors="pt")
    return encoding


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('phishing-detector')
model.eval()  # Set the model to evaluation mode


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
import torch

def predict_email_category(email_text):
    # Preprocess the email text
    encoding = preprocess_email(email_text)
    
    # Ensure the input is moved to the correct device (e.g., CUDA or CPU)
    input_ids = encoding['input_ids'].to('cuda') if torch.cuda.is_available() else encoding['input_ids'].to('cpu')
    attention_mask = encoding['attention_mask'].to('cuda') if torch.cuda.is_available() else encoding['attention_mask'].to('cpu')
    
    # Perform inference
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    
    # Get the predicted class
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    
    # Return the predicted class
    return predicted_class

# Example usage
email_text = "Congratulations! You've won a $1000 gift card. Click here to claim your prize."
predicted_class = predict_email_category(email_text)

# Map predicted class back to label
label_mapping = {0: 'Ham', 1: 'Spam'}
predicted_label = label_mapping[predicted_class]

print(f"Predicted category: {predicted_label}")


Predicted category: Spam
